# SteganoGAN in Keras
This notebook contains code attempting to reimplement SteganoGAN in Keras, for the purpose of better understanding (and scrutinizing) it.

*Based on https://github.com/DAI-Lab/SteganoGAN/tree/master/steganogan*

### Modules

In [1]:
import os
os.environ["KERAS_BACKEND"] = "tensorflow"

import sys
sys.path.append("..")

import tensorflow as tf
import numpy as np
from keras.losses import BinaryCrossentropy

from models import (
  steganogan_encoder_basic_model,
  steganogan_encoder_residual_model,
  steganogan_encoder_dense_model,

  steganogan_decoder_basic_model,
  steganogan_decoder_dense_model,
  
  steganogan_critic_model
)
from keras_steganogan_basic import KerasSteganoGAN

2024-07-16 10:33:13.875805: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


### Constants

In [2]:
# Image dimensions
IMAGE_HEIGHT = 128
IMAGE_WIDTH = 128
IMAGE_CHANNELS = 3

IMAGE_SHAPE = (IMAGE_HEIGHT, IMAGE_WIDTH, IMAGE_CHANNELS)
MESSAGE_DEPTH = 2
BATCH_SIZE = 4
MODEL_PATH = '../pretrained_models/steganoGAN_basic_enc_basic_dec.keras'

----

### Call main encode and decode functions (with creating steganographic image and decoding it)

In [9]:
steganoGAN = KerasSteganoGAN(image_shape=IMAGE_SHAPE, data_depth=MESSAGE_DEPTH, model_path=MODEL_PATH)

steganoGAN.encode('../images/testing/input1.png', '../images/basic/output1.png', 'Hello, World! 1111 asdf')
print(steganoGAN.decode('../images/basic/output1.png'))

steganoGAN.encode('../images/testing/input2.png', '../images/basic/output2.png', 'Hello, World! 2222 asdf')
print(steganoGAN.decode('../images/basic/output2.png'))

steganoGAN.encode('../images/testing/input3.png', '../images/basic/output3.png', 'Hello, World! 3333 asdf ')
print(steganoGAN.decode('../images/basic/output3.png'))

steganoGAN.encode('../images/testing/input4.png', '../images/basic/output4.png', 'Hello, World! 4444 asdf')
print(steganoGAN.decode('../images/basic/output4.png'))

Found 16 candidates for message, choosing most common.
Hello, World! 1111
Found 28 candidates for message, choosing most common.
Hello, World! 2222
Found 41 candidates for message, choosing most common.
Hello, World! 3333
Found 7 candidates for message, choosing most common.
Hello, World! 4444


### SteganoGAN predict random data with metrics

In [4]:
cover_image = tf.random.uniform([1, IMAGE_HEIGHT, IMAGE_WIDTH, IMAGE_CHANNELS], -1, 1, dtype=tf.float32)
message = tf.cast(tf.random.uniform([1, IMAGE_HEIGHT, IMAGE_WIDTH, MESSAGE_DEPTH], 0, 2, dtype=tf.int32), tf.float32)

stego_img, recovered_msg = steganoGAN.predict([cover_image, message])

print("stego_img min: {0}, max: {1}".format(tf.reduce_min(stego_img), tf.reduce_max(stego_img)))
print("recovered_msg min: {0}, max: {1}".format(tf.reduce_min(recovered_msg), tf.reduce_max(recovered_msg)))

print("BinaryCrossentropy: {0}".format(BinaryCrossentropy(from_logits=False)(message, recovered_msg)))
print("PSNR: {0}".format(tf.reduce_mean(tf.image.psnr(cover_image, stego_img, 1))))
print("SSIM: {0}".format(tf.reduce_mean(tf.image.ssim(cover_image, stego_img, 1))))

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 284ms/step
stego_img min: -0.8220856785774231, max: 0.7682651877403259
recovered_msg min: 0.0, max: 1.0
BinaryCrossentropy: 0.1870475709438324
PSNR: 6.1603007316589355
SSIM: 0.18809054791927338


----

### Testing different functions 

In [5]:
# Create two random tensors
tensor1 = tf.random.uniform((4, 128, 128, 2), minval=0, maxval=1, dtype=tf.int32)
tensor2 = tf.random.uniform((4, 128, 128, 2), minval=0, maxval=1, dtype=tf.float32)

# Calculate binary crossentropy
loss = BinaryCrossentropy(from_logits=False)
loss = loss(tf.constant([0, 0, 0, 0]), tf.constant([1, 1, 1, 1]))
#loss = loss(tensor1, tensor2)

loss.numpy().astype(np.uint8)

15

In [6]:
x = tf.constant([-1, -15, 0, 5])

ones = tf.ones(tf.shape(x))
zeros = tf.zeros(tf.shape(x))

tf.keras.backend.switch(x > 0, ones, zeros)

<tf.Tensor: shape=(4,), dtype=float32, numpy=array([0., 0., 0., 1.], dtype=float32)>

In [7]:
x = tf.constant([0, 0, 0, 0])
y = tf.constant([1, 1, 1, 1])

mse = tf.keras.losses.MeanSquaredError() 
mse(x, y).numpy().astype(np.uint8)

1